# ANÁLISE DE COMBINAÇÕES - Definir Hierarquias

Objetivo: Ver que combinações de códigos existem no mesmo dia/colaborador APÓS remover incompatibilidades, para definir regras de prioridade.

In [ ]:
import pandas as pd
import numpy as np

print('Carregando dados...')

In [ ]:
# 1. Carregar dados
df_raw = pd.read_csv('combined_data.csv')
df_raw['Data'] = pd.to_datetime(df_raw['Data'])

print(f'Total registos: {len(df_raw):,}')
print(f'Colaboradores: {df_raw["login_colaborador"].nunique():,}')

In [ ]:
# 2. Aplicar nova classificação
df_codigos = pd.read_excel('códigos_V2.xlsx')

df = df_raw.merge(
    df_codigos,
    left_on='segmento_processado_codigo',
    right_on='Codigo Segmento',
    how='left'
)

print(f'✓ Classificação aplicada')
print(f'Registos com classificação: {df["Nivel 1"].notna().sum():,}')

In [ ]:
# 3. REMOVER INCOMPATIBILIDADES (copiar lógica do notebook principal)
print('Criando matriz de compatibilidade...')

# Matriz de compatibilidade Nivel 2
compat_matrix = pd.DataFrame([
    # Trabalho Pago
    ['Presença', 'Atraso', 1],
    ['Presença', 'Formação', 1],
    
    # Incompatibilidades principais
    ['Presença', 'Ausência Médica', 0],
    ['Presença', 'Ausência Injustificada', 0],
    ['Presença', 'Licença Mat/Pat', 0],
    ['Presença', 'Férias', 0],
    ['Presença', 'Falta Justificada', 0],
    
    ['Trabalho Pago', 'Ausência Médica', 0],
    ['Trabalho Pago', 'Ausência Injustificada', 0],
    ['Trabalho Pago', 'Falta Injustificada', 0],
], columns=['cat1', 'cat2', 'compativel'])

compat_dict = {}
for _, row in compat_matrix.iterrows():
    key1 = tuple(sorted([row['cat1'], row['cat2']]))
    compat_dict[key1] = row['compativel']

print(f'✓ Matriz criada: {len(compat_dict)} regras')

In [ ]:
# 4. Identificar dias duplicados
print('Identificando dias com múltiplos registos...')

dias_duplicados = df.groupby(['login_colaborador', 'Data']).size()
dias_duplicados = dias_duplicados[dias_duplicados > 1].reset_index()
dias_duplicados.columns = ['login_colaborador', 'Data', 'num_registos']

print(f'Dias com múltiplos registos: {len(dias_duplicados):,}')

In [ ]:
# 5. Testar incompatibilidades (VERSÃO OTIMIZADA)
print('Testando incompatibilidades nos dias duplicados...\n')

# PRÉ-FILTRAR apenas dias duplicados
dias_dup_list = list(zip(dias_duplicados['login_colaborador'], dias_duplicados['Data']))
df_dup = df[df.set_index(['login_colaborador', 'Data']).index.isin(dias_dup_list)].copy()

print(f'   Registos a testar: {len(df_dup):,}')

# PRÉ-AGRUPAR dados (UMA vez!)
df_dup_grouped = df_dup.groupby(['login_colaborador', 'Data']).agg({
    'Nivel 2': lambda x: list(x.dropna().unique()),
    'segmento_processado_codigo': lambda x: list(x.unique()),
    'nome_colaborador': 'first'
}).reset_index()

df_dup_grouped.columns = ['login_colaborador', 'Data', 'categorias_nivel2', 'codigos', 'nome']

print(f'   Dias únicos a testar: {len(df_dup_grouped):,}')

# Identificar incompatibilidades
print('\nTestando pares de categorias...')
dias_incompativeis = []

for idx, row in df_dup_grouped.iterrows():
    if idx % 5000 == 0 and idx > 0:
        print(f'   Processados {idx:,}/{len(df_dup_grouped):,} dias...')

    login = row['login_colaborador']
    data = row['Data']
    categorias = row['categorias_nivel2']
    codigos = row['codigos']
    nome = row['nome']

    # Testar todos os pares
    incompativel_encontrado = False
    pares_incompativeis = []

    for i, cat1 in enumerate(categorias):
        for cat2 in categorias[i+1:]:
            key = tuple(sorted([cat1, cat2]))
            if key in compat_dict and compat_dict[key] == 0:
                incompativel_encontrado = True
                pares_incompativeis.append(f'{cat1} + {cat2}')

    if incompativel_encontrado:
        dias_incompativeis.append({
            'login_colaborador': login,
            'Data': data,
            'nome_colaborador': nome,
            'categorias': ', '.join(categorias),
            'codigos': ', '.join(codigos),
            'pares_incompativeis': ' | '.join(pares_incompativeis)
        })

df_incompativeis = pd.DataFrame(dias_incompativeis)

print(f'\n✓ Teste concluído')
print(f'🔴 INCOMPATIBILIDADES ENCONTRADAS: {len(df_incompativeis)}')

if len(df_incompativeis) > 0:
    print(f'\nPrimeiros 10 casos:')
    print(df_incompativeis[['Data', 'nome_colaborador', 'pares_incompativeis']].head(10))


In [ ]:
# 6. REMOVER incompatibilidades
print('Removendo dias incompatíveis...')

if len(df_incompativeis) > 0:
    df_limpo = df.copy()
    
    for _, incompat in df_incompativeis.iterrows():
        mask = (df_limpo['login_colaborador'] == incompat['login_colaborador']) & \
               (df_limpo['Data'] == incompat['Data'])
        df_limpo = df_limpo[~mask]
    
    print(f'✓ Removidos {len(df) - len(df_limpo):,} registos')
    print(f'✓ Dataset limpo: {len(df_limpo):,} registos')
else:
    df_limpo = df.copy()
    print('✓ Nenhuma incompatibilidade - dataset limpo')

---

## ANÁLISE DE COMBINAÇÕES

Agora que removemos incompatibilidades, vamos ver que combinações RESTAM.

In [ ]:
# 7. Identificar dias AINDA com múltiplos registos (após limpeza)
print('=== COMBINAÇÕES APÓS REMOVER INCOMPATIBILIDADES ===\n')

dias_multiplos = df_limpo.groupby(['login_colaborador', 'Data']).size()
dias_multiplos = dias_multiplos[dias_multiplos > 1].reset_index()
dias_multiplos.columns = ['login_colaborador', 'Data', 'num_registos']

print(f'Dias com múltiplos registos (compatíveis): {len(dias_multiplos):,}')
print(f'Total registos nesses dias: {dias_multiplos["num_registos"].sum():,}')

In [ ]:
# 8. EXTRAIR todas as combinações que existem
print('\nExtraindo combinações de Nivel 1...\n')

combinacoes = []

for _, dia in dias_multiplos.iterrows():
    login = dia['login_colaborador']
    data = dia['Data']
    
    registos_dia = df_limpo[(df_limpo['login_colaborador'] == login) & (df_limpo['Data'] == data)]
    
    # Pegar Nivel 1
    nivel1_list = registos_dia['Nivel 1'].dropna().unique().tolist()
    nivel2_list = registos_dia['Nivel 2'].dropna().unique().tolist()
    
    # Criar string ordenada
    nivel1_str = ' + '.join(sorted(nivel1_list))
    nivel2_str = ' + '.join(sorted(nivel2_list))
    
    combinacoes.append({
        'login_colaborador': login,
        'Data': data,
        'num_registos': len(registos_dia),
        'combinacao_nivel1': nivel1_str,
        'combinacao_nivel2': nivel2_str
    })

df_combinacoes = pd.DataFrame(combinacoes)

print(f'Total de casos com combinações: {len(df_combinacoes):,}')

In [ ]:
# 9. CONTAR frequência de cada combinação (Nivel 1)
print('\n📊 FREQUÊNCIA DAS COMBINAÇÕES (NIVEL 1):\n')

freq_nivel1 = df_combinacoes['combinacao_nivel1'].value_counts()

print(f'Total de combinações únicas: {len(freq_nivel1)}\n')
print('Top 20 combinações mais frequentes:\n')

for idx, (comb, count) in enumerate(freq_nivel1.head(20).items(), 1):
    pct = count / len(df_combinacoes) * 100
    print(f'{idx:2d}. {comb:60s}: {count:6,} casos ({pct:5.2f}%)')

In [ ]:
# 10. CONTAR frequência de cada combinação (Nivel 2)
print('\n📊 FREQUÊNCIA DAS COMBINAÇÕES (NIVEL 2):\n')

freq_nivel2 = df_combinacoes['combinacao_nivel2'].value_counts()

print(f'Total de combinações únicas: {len(freq_nivel2)}\n')
print('Top 20 combinações mais frequentes:\n')

for idx, (comb, count) in enumerate(freq_nivel2.head(20).items(), 1):
    pct = count / len(df_combinacoes) * 100
    print(f'{idx:2d}. {comb:60s}: {count:6,} casos ({pct:5.2f}%)')

In [ ]:
# 11. Exportar para Excel para análise detalhada
print('\nExportando para Excel...\n')

# Sheet 1: Resumo de frequências Nivel 1
freq_nivel1_df = freq_nivel1.reset_index()
freq_nivel1_df.columns = ['Combinação Nivel 1', 'Frequência']
freq_nivel1_df['Percentagem'] = (freq_nivel1_df['Frequência'] / len(df_combinacoes) * 100).round(2)

# Sheet 2: Resumo de frequências Nivel 2
freq_nivel2_df = freq_nivel2.reset_index()
freq_nivel2_df.columns = ['Combinação Nivel 2', 'Frequência']
freq_nivel2_df['Percentagem'] = (freq_nivel2_df['Frequência'] / len(df_combinacoes) * 100).round(2)

# Sheet 3: Detalhes de todos os casos
df_combinacoes_export = df_combinacoes.merge(
    df_limpo[['login_colaborador', 'Data', 'nome_colaborador']].drop_duplicates(),
    on=['login_colaborador', 'Data'],
    how='left'
)

# Exportar
with pd.ExcelWriter('analise_combinacoes.xlsx', engine='openpyxl') as writer:
    freq_nivel1_df.to_excel(writer, sheet_name='Frequencias_Nivel1', index=False)
    freq_nivel2_df.to_excel(writer, sheet_name='Frequencias_Nivel2', index=False)
    df_combinacoes_export.to_excel(writer, sheet_name='Detalhes', index=False)

print('✓ Exportado: analise_combinacoes.xlsx')
print('\n  Sheet 1: Frequencias_Nivel1 - Resumo por Nivel 1')
print('  Sheet 2: Frequencias_Nivel2 - Resumo por Nivel 2')
print('  Sheet 3: Detalhes - Todos os casos com combinações')

---

## PRÓXIMO PASSO

Abre o ficheiro **analise_combinacoes.xlsx** e define as **regras de prioridade** para cada combinação encontrada.

Exemplo:
- `Atraso + Trabalho Pago` → Para absentismo: Trabalho Pago | Para atrasos: Atraso
- `Ausência + Trabalho Pago` → ??? (define a regra)
- etc.